# 1- Imports and Warning Suppression Setup


Explanation:
Imports the warnings module and disables Python warnings so that no warning messages appear in the output during execution.

If removed:
You may see warnings from libraries (like deprecation warnings) that could clutter the terminal, but the code will usually still run normally.


**Explanation for each library**:

- `cv2`: OpenCV, used for video processing, drawing on frames, and displaying windows.

- `mediapipe`: A ready-made library for hand tracking (landmarks).

- `numpy`: For array operations and handling images as arrays.

- `joblib`: For loading the saved model (pickled model).

- `arabic_reshaper + bidi.algorithm.get_display`: Reconstructs Arabic characters and fixes direction for correct display (used to render Arabic text with PIL/OpenCV).

- `PIL` (ImageFont, ImageDraw, Image): Used to draw Arabic text on images because OpenCV does not support Arabic rendering properly.

**If any of these are removed**:
The part of the code depending on that library will fail (e.g., without joblib you can't load the model; without arabic_reshaper, Arabic text will appear disjointed).

---
---

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import cv2
import mediapipe as mp
import numpy as np
import joblib
import arabic_reshaper
from bidi.algorithm import get_display
from PIL import ImageFont, ImageDraw, Image


## 2-  General Settings / Constant Variables

**Explanation**:

- `MODEL_PATH`: Path to the saved model file (pickle).

- `THRESHOLD_FRAMES`: Number of frames required to confirm a predicted letter before adding it — prevents capturing multiple times from a single frame.

- `SPACE_COOLDOWN_MAX`: Number of cooldown frames after adding a space (SPACE) before allowing another one.

- `myCamere`: Camera index (0 is usually the default webcam).

# UI settings:
Top/bottom bar height and minimum window width (to avoid overlap in UI elements).

**If changed/removed**:

- Lowering `THRESHOLD_FRAMES` → faster but more prone to errors.

- Increasing it → slower input but more stable.

- Setting `MIN_WINDOW_WIDTH` too small → text may overlap or collide with UI elements.

---
---

In [ ]:
# 1. الإعدادات
# =============================================================
MODEL_PATH = r"linearSVC.pkl"
THRESHOLD_FRAMES = 25   
SPACE_COOLDOWN_MAX = 40 
myCamere = 0

# إعدادات أبعاد الـ UI (بالبكسل)
UI_TOP_HEIGHT = 50      # ارتفاع الشريط العلوي
UI_BOTTOM_HEIGHT = 80  # ارتفاع الشريط السفلي
MIN_WINDOW_WIDTH = 800 # 👈 عرض النافذة الثابت (لضمان عدم تداخل النص)

## 3- State Variables


**Explanation**: Variables that store the current state:

- `sentence`: The actual constructed text.

- `last_prediction`: The last predicted letter — used to check stability across frames.

- `frame_counter`: Counts how many consecutive frames kept the same prediction.

- `space_cooldown`: Cooldown counter for adding a SPACE.

- `writing_enabled`: Whether writing mode is enabled or not.

**If removed**:
Text tracking and input stabilization would break or behave incorrectly.


---
---


In [ ]:
# متغيرات النظام
sentence = ""
last_prediction = None
frame_counter = 0
space_cooldown = 0
writing_enabled = False 

## 4- Loading the Model


**Explanation**:

- Attempts to load the file `linearSVC.pkl`.

- Some people store models in a dictionary like `{'model': model, ...}`, so this code handles both possibilities.

- If loading fails, it prints an error and exits the program (`exit()`).

**If `try/except` is removed**:
A loading error would raise an exception and crash the program in a less friendly way.

**Note**:
During development, you may prefer printing the actual error cause:
`except Exception as e: print(e)`


---
---


In [ ]:
print("⏳ Loading Model...")
try:
    loaded_data = joblib.load(MODEL_PATH)
    if isinstance(loaded_data, dict) and 'model' in loaded_data:
        model = loaded_data['model']
    else:
        model = loaded_data
    print("✅ Model Loaded.")
except Exception as e:
    print("❌ Error loading model.")
    print(e)
    exit()

## 5- Helper Functions

`normalize_char`

**Explanation**:
Normalizes certain Arabic letters into a unified form `(here it converts "أ" to "ا")`.

**If removed**:
Visual variations may appear (e.g., `"أ"` and `"ا"` would be treated as different characters).

---

In [ ]:
def normalize_char(char):
    if char == 'أ': return 'ا'
    return char


`extract_features`


**Explanation**:
Converts Mediapipe `hand_landmarks` coordinates into a feature list relative to a base point (index 0).

- Assumes `points[0]` is the reference point (usually the wrist).

- Returns coordinate differences for each landmark — producing a representation independent of camera distance/position.

**If removed/changed**:
The model expects features in a specific format.
Changing the extraction method will break compatibility with the trained model.
Using raw coordinates without normalization would make the model sensitive to distance and hand position.

---

In [ ]:
def extract_features(hand_landmarks):
    points = []
    for lm in hand_landmarks.landmark:
        points.append([lm.x, lm.y, lm.z])
    base_x, base_y, base_z = points[0]
    final_features = []
    for i in range(1, len(points)):
        p = points[i]
        final_features.extend([p[0] - base_x, p[1] - base_y, p[2] - base_z])
    return final_features

`draw_arabic_text`


**Explanation**:
Draws Arabic text on an image (numpy array) using PIL:

- Converts the numpy image to a PIL image.

- Uses `arabic_reshaper` + `bidi.get_display` to render Arabic letters correctly.

Draws the text using the chosen font, falling back to default if `arial.ttf` is missing.

- Returns the modified image as a numpy array.

**If removed**:
Displaying Arabic text using `cv2.putText` results in broken or reversed letters.
Arabic shaping and direction handling will not work.
This function is essential for proper Arabic rendering.

---
---

In [ ]:
def draw_arabic_text(img, text, position, color=(0, 255, 0), font_size=32):
    img_pil = Image.fromarray(img)
    reshaped_text = arabic_reshaper.reshape(text)
    bidi_text = get_display(reshaped_text)
    draw = ImageDraw.Draw(img_pil)
    try:
        font = ImageFont.truetype("arial.ttf", font_size)
    except:
        font = ImageFont.load_default()
    position = (int(position[0]), int(position[1]))
    draw.text(position, bidi_text, font=font, fill=color)
    return np.array(img_pil)

## 6- Mediapipe Setup

**Explanation**:

- `mp_hands` and `mp_drawing` provide hand-tracking and drawing utilities.

- `Hands` is configured for:

- Video input (not static images),

- Up to 2 hands,

- Confidence thresholds for detection and tracking.

**If values are changed**:

- Lower `min_detection_confidence` → more false positives.

- Higher values → may fail to detect hands in poor lighting.

- `max_num_hands` > 2 is unnecessary in most cases.

---

In [ ]:
# 3. إعداد Mediapipe
# =============================================================
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.7, min_tracking_confidence=0.7)

## 7- Camera Initialization & Resolution Setup
```py
cap = cv2.VideoCapture(myCamere)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
```

**Explanation**:
Opens your device camera (using index `myCamere`) and attempts to set the resolution to 1280×720.

If `cap.set` is removed:
The camera will run at its default resolution (usually lower).
Note: Some cameras cannot support the requested resolution and will fallback to what they support.


---
---

## 8- The Main Loop
```py
while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
```

**Explanation**:
A loop that keeps capturing frames as long as the camera is open.

If `ret` is `False`, this means a capture error or end of stream → break.

```py
h_cam, w_cam, _ = frame.shape
rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
results = hands.process(rgb_frame)
```

**Explanation**:

- Retrieves the frame dimensions.

- Converts BGR → RGB because Mediapipe expects RGB.

- `hands.process` returns the detected hand landmarks.

**If RGB conversion is forgotten**:
Mediapipe may perform poorly or give incorrect landmark detection because it would receive incorrectly ordered color channels.


---
---


## 9- Hand Processing (If results are found)
```py
if results.multi_hand_landmarks:
    hands_count = len(results.multi_hand_landmarks)
```

**Explanation**:
If one or more hands are detected, process them depending on how many hands are found.

---

**Case: Two Hands Detected (`hands_count == 2`)**
```py
if hands_count == 2:
    frame_counter = 0
    if writing_enabled and space_cooldown == 0:
        sentence += " "
        space_cooldown = SPACE_COOLDOWN_MAX

    cv2.putText(frame, "SPACE", (50, h_cam - 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
```

**Explanation**:

Two hands are treated as the gesture for *SPACE*. When this occurs:

- Reset `frame_counter` (to avoid recording a letter at this moment).

- If writing is enabled and there is no cooldown, add a space to `sentence` and start the cooldown timer.

- Display `"SPACE"` on the frame.

- Draw landmarks for both hands.

**If removed**:
The system will not detect the SPACE gesture (2-hand gesture), will not insert spaces, and will not display `"SPACE"`.

---
**Case: One Hand Detected (`hands_count == 1`)**
```py
elif hands_count == 1:
    hand_landmarks = results.multi_hand_landmarks[0]
    mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    
    data = extract_features(hand_landmarks)
    raw_char = "?"
    try:
        prediction = model.predict([data])[0]
        raw_char = str(prediction)
    except: pass

    current_char = normalize_char(raw_char)
```

---
**Step-by-step explanation:**

- Obtain the single hand's landmarks and draw them.

- Extract features using `extract_features`.

- Feed the features to the model to predict the corresponding character.

- If prediction fails (incorrect data shape, etc.), ignore and keep `"?"`.

- Normalize Arabic characters using `normalize_char`.

**If the try/except is removed**:
If the model throws any error, the entire loop would crash.
The `try` ensures the app keeps running even when a prediction occasionally fails.

---

**Frame Stability Logic (Debouncing)**
```py
if current_char == last_prediction:
    frame_counter += 1
else:
    frame_counter = 0
    last_prediction = current_char

if frame_counter >= THRESHOLD_FRAMES:
    if writing_enabled:
        sentence += current_char
        frame_counter = 0
    else:
        frame_counter = THRESHOLD_FRAMES
```

**Explanation**:

- Compares the current prediction with the last one.

- If the same prediction persists for `THRESHOLD_FRAMES`, it is considered stable.

- Only then is the character added to `sentence` (if writing is enabled).

This avoids recording letters from noisy or unstable frames.

**If removed**:
The system would add a letter immediately on every prediction → extremely noisy, unstable, and full of repeated letters.

---

**Using Hand Position for Drawing (Progress Bar & Character Box)**
```py
cx, cy = int(hand_landmarks.landmark[0].x * w_cam), int(hand_landmarks.landmark[0].y * h_cam)
bar_width = int((frame_counter / THRESHOLD_FRAMES) * 100)
bar_color = (0, 255, 0) if writing_enabled else (0, 200, 255)

bar_y = cy + 40 
cv2.rectangle(frame, (cx-50, bar_y), (cx-50+100, bar_y+10), (50, 50, 50), -1)
cv2.rectangle(frame, (cx-50, bar_y), (cx-50+bar_width, bar_y+10), bar_color, -1)

box_y_start = bar_y + 15
box_y_end = box_y_start + 50
cv2.rectangle(frame, (cx-50, box_y_start), (cx+50, box_y_end), (0, 0, 0), -1)
frame = draw_arabic_text(frame, current_char, (cx-10, box_y_start + 5), (255, 255, 0))
```

**Explanation**:

- Computes `(cx, cy)` from the wrist landmark (reference point).

- Draws a progress bar showing how close the stability check is to completing.

- Draws a black box showing the current detected character.

- Uses `draw_arabic_text` to display Arabic properly.

**If removed**:
The user would not see progress or character feedback on the hand → a much worse interactive experience.


---
---

## 10- SPACE Cooldown
```py
if space_cooldown > 0: space_cooldown -= 1
```

**Explanation**:
Decrements the cooldown counter every frame until it reaches zero.
This prevents adding multiple spaces rapidly in consecutive frames.

**If removed**:
When the SPACE gesture appears once, multiple spaces could be added quickly as long as two hands stay detected.

---
---

## 11- Building a Wide Display Canvas
```py
final_ui_width = max(w_cam, MIN_WINDOW_WIDTH)
total_height = UI_TOP_HEIGHT + h_cam + UI_BOTTOM_HEIGHT
canvas = np.zeros((total_height, final_ui_width, 3), dtype=np.uint8)
canvas[:] = (200, 80, 50)

x_offset = (final_ui_width - w_cam) // 2
canvas[UI_TOP_HEIGHT : UI_TOP_HEIGHT + h_cam, x_offset : x_offset + w_cam] = frame
```

**Explanation**:

- Creates a display canvas wider than the camera frame (to include a top bar, bottom bar, and text).

- `canvas[:] = (200, 80, 50)` fills the background with a solid color.

- Centers the camera frame inside the canvas using `x_offset`.

**If removed**:
Displaying only `frame` would remove the top UI bar, the bottom text area, and all layout formatting.

---
**Drawing Mode Status & Instructions**
```py
if writing_enabled:
    mode_text = "MODE: WRITING (ON)"
    mode_color = (0, 225, 0)
else:
    mode_text = "MODE: PREVIEW (OFF)"
    mode_color = (0, 100, 255)

cv2.putText(canvas, mode_text, (20, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.8, mode_color, 2)
instructions = "S:Write | C:Clear | Back:Undo | Q:Quit"
(text_w, _), _ = cv2.getTextSize(instructions, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
cv2.putText(canvas, instructions, (final_ui_width - text_w - 20, 35), cv2.FONT_HERSHEY_SIMPLEX, .8, (200, 250, 250), 1)
cv2.line(canvas, (0, UI_TOP_HEIGHT), (final_ui_width, UI_TOP_HEIGHT), (50, 50, 50), 2)
```

**Explanation**:
Shows the mode status (writing or preview) and keyboard instructions in the top UI bar, then draws a separator line.

**If removed**:
The user would not know the current writing mode or the available shortcuts.

---

**Bottom Bar & Displaying the Translated Text**
```py
bottom_bar_y_start = UI_TOP_HEIGHT + h_cam
cv2.line(canvas, (0, bottom_bar_y_start), (final_ui_width, bottom_bar_y_start), (50, 50, 50), 2)

text_y_pos = bottom_bar_y_start + 15
canvas = draw_arabic_text(canvas, "النص: " + sentence, (20, text_y_pos), (255, 255, 255), font_size=40)
```

**Explanation**:
Draws a horizontal line above the bottom bar, then displays the constructed text using `draw_arabic_text` (to support proper Arabic rendering).

**If removed**:
The final translated/generated sentence would not appear at the bottom of the screen.


---
---

## 12- Display, Keyboard Controls, and Exit Logic
```py
cv2.imshow('Tech mind - Sign Language Translator', canvas)
key = cv2.waitKey(1) & 0xFF

if key == ord('q'): break
elif key == 8: sentence = sentence[:-1]
elif key == ord('s'): writing_enabled = not writing_enabled
elif key == ord('c'): sentence = ""
```

**Explanation**:

- Displays the window with a custom title.

- `waitKey(1)` reads keyboard input.

**Keyboard controls**:

| Key | Function |
| :---: | :--- |
| **S** | Enable/Disable writing mode (Start/Stop) |
| **C** | Clear the entire sentence |
| **Back/8** | Delete the last character |
| **Q** | Quit the program |


**Notes**:

- Using key code 8 for Backspace depends on OpenCV + OS; on some systems you may need `127` or `ord('\b')`.

- Without `waitKey`, the program wouldn't update the window or read keyboard input.


---
---


## 13- Cleaning Up Resources
```py
cap.release()
cv2.destroyAllWindows()
```

**Explanation**:
Releases the camera and closes all OpenCV windows.

**If removed**:
The camera may remain locked, making it unusable for other applications; windows may stay open.

In [ ]:
# =============================================================
# 4. التشغيل
# =============================================================
cap = cv2.VideoCapture(myCamere) 
# نحاول طلب دقة عالية، لكن الكود سيتكيف مهما كانت النتيجة
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break

    # أبعاد الفيديو الأصلي (من الكاميرا)
    h_cam, w_cam, _ = frame.shape

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    
    # --- معالجة اليدين (كل الإحداثيات هنا خاصة بإطار الكاميرا فقط) ---
    if results.multi_hand_landmarks:
        hands_count = len(results.multi_hand_landmarks)

        if hands_count == 2:
            frame_counter = 0
            if writing_enabled and space_cooldown == 0:
                sentence += " "
                space_cooldown = SPACE_COOLDOWN_MAX
            
            # رسم كلمة SPACE
            cv2.putText(frame, "SPACE", (50, h_cam - 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        elif hands_count == 1:
            hand_landmarks = results.multi_hand_landmarks[0]
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            
            data = extract_features(hand_landmarks)
            raw_char = "?"
            try:
                prediction = model.predict([data])[0]
                raw_char = str(prediction)
            except: pass

            current_char = normalize_char(raw_char)

            if current_char == last_prediction:
                frame_counter += 1
            else:
                frame_counter = 0
                last_prediction = current_char

            if frame_counter >= THRESHOLD_FRAMES:
                if writing_enabled:
                    sentence += current_char
                    frame_counter = 0 
                else:
                    frame_counter = THRESHOLD_FRAMES 

            # حساب المكان بالنسبة لإطار الكاميرا
            cx, cy = int(hand_landmarks.landmark[0].x * w_cam), int(hand_landmarks.landmark[0].y * h_cam)
            
            # الرسم على إطار الكاميرا
            bar_width = int((frame_counter / THRESHOLD_FRAMES) * 100)
            bar_color = (0, 255, 0) if writing_enabled else (0, 200, 255)
            
            bar_y = cy + 40 
            cv2.rectangle(frame, (cx-50, bar_y), (cx-50+100, bar_y+10), (50, 50, 50), -1)
            cv2.rectangle(frame, (cx-50, bar_y), (cx-50+bar_width, bar_y+10), bar_color, -1)
            
            box_y_start = bar_y + 15
            box_y_end = box_y_start + 50
            cv2.rectangle(frame, (cx-50, box_y_start), (cx+50, box_y_end), (0, 0, 0), -1)
            frame = draw_arabic_text(frame, current_char, (cx-10, box_y_start + 5), (255, 255, 0))

    if space_cooldown > 0: space_cooldown -= 1

    # =========================================================
    # بناء الواجهة العريضة (Canvas)
    # =========================================================
    
    # 1. تحديد عرض النافذة النهائي (إما العرض الثابت 1280 أو عرض الكاميرا أيهما أكبر)
    final_ui_width = max(w_cam, MIN_WINDOW_WIDTH)
    
    # 2. إنشاء لوحة سوداء
    total_height = UI_TOP_HEIGHT + h_cam + UI_BOTTOM_HEIGHT
    canvas = np.zeros((total_height, final_ui_width, 3), dtype=np.uint8)
    canvas[:] = (200, 80, 50)
    
    # 3. حساب نقطة البداية لتوسيط الكاميرا
    x_offset = (final_ui_width - w_cam) // 2
    
    # 4. وضع إطار الكاميرا في منتصف الـ Canvas
    canvas[UI_TOP_HEIGHT : UI_TOP_HEIGHT + h_cam, x_offset : x_offset + w_cam] = frame

    # --- رسم النصوص والأزرار على الـ Canvas العريض ---
    if writing_enabled:
        mode_text = "MODE: WRITING (ON)"
        mode_color = (0, 225, 0) 
    else:
        mode_text = "MODE: PREVIEW (OFF)"
        mode_color = (0, 100, 255)

    # الشريط العلوي
    cv2.putText(canvas, mode_text, (20, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.8, mode_color, 2)
    
    instructions = "S:Write | C:Clear | Back:Undo | Q:Quit"
    (text_w, _), _ = cv2.getTextSize(instructions, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
    # نستخدم final_ui_width للتأكد ان التعليمات في اقصى اليمين للشاشة العريضة
    cv2.putText(canvas, instructions, (final_ui_width - text_w - 20, 35), cv2.FONT_HERSHEY_SIMPLEX, .8, (200, 250, 250), 1)

    cv2.line(canvas, (0, UI_TOP_HEIGHT), (final_ui_width, UI_TOP_HEIGHT), (50, 50, 50), 2)

    # الشريط السفلي
    bottom_bar_y_start = UI_TOP_HEIGHT + h_cam
    cv2.line(canvas, (0, bottom_bar_y_start), (final_ui_width, bottom_bar_y_start), (50, 50, 50), 2)
    
    # كتابة النص المترجم
    text_y_pos = bottom_bar_y_start + 15
    # النص الآن يملك مساحة 1280 بيكسل كاملة
    canvas = draw_arabic_text(canvas, "النص: " + sentence, (20, text_y_pos), (255, 255, 255), font_size=40)

    # عرض النتيجة النهائية
    cv2.imshow('Tech mind - Sign Language Translator', canvas)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'): break
    elif key == 8: sentence = sentence[:-1] 
    elif key == ord('s'): 
        writing_enabled = not writing_enabled
    elif key == ord('c'): 
        sentence = ""
        print(">> Cleared All Text")

cap.release()
cv2.destroyAllWindows()